In [11]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def edge_promoting(root, save):
    file_list = os.listdir(root)
    if not os.path.isdir(save):
        os.makedirs(save)
    kernel_size = 5
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    gauss = cv2.getGaussianKernel(kernel_size, 0)
    gauss = gauss * gauss.transpose(1, 0)
    n = 1
    for f in tqdm(file_list):
        if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # 이미지 파일 경로 설정
            input_path = os.path.join(root, f)
            output_path = os.path.join(save, str(n) + '.png')
            
            # 이미지 읽기
            rgb_img = cv2.imread(input_path)
            gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
            pad_img = np.pad(rgb_img, ((3,3), (3,3), (0,0)), mode='reflect')
            
            # 이미지 가장자리 검출
            edges = cv2.Canny(gray_img, 150, 500)
            
            # 이미지 dilation 적용
            dilation = cv2.dilate(edges, kernel)
            
            # 가우시안 블러 적용
            gauss_img = np.copy(rgb_img)
            idx = np.where(dilation != 0)
            for i in range(np.sum(dilation != 0)):
                row, col = idx[0][i], idx[1][i]
                row_padded, col_padded = row + 3, col + 3
                
                gauss_img[row, col, 0] = np.sum(np.multiply(pad_img[row_padded-2:row_padded+3, col_padded-2:col_padded+3, 0], gauss))
                gauss_img[row, col, 1] = np.sum(np.multiply(pad_img[row_padded-2:row_padded+3, col_padded-2:col_padded+3, 1], gauss))
                gauss_img[row, col, 2] = np.sum(np.multiply(pad_img[row_padded-2:row_padded+3, col_padded-2:col_padded+3, 2], gauss))
            
            # 결과 이미지 저장
            cv2.imwrite(output_path, gauss_img)
            n += 1

# 함수 호출
edge_promoting("output_images_2_crop", "output_images_2_crop_edge_blur")






100%|██████████| 4622/4622 [30:11<00:00,  2.55it/s]  
